### Problem

Assume that you need to optimize a manufacturing company’s supply chain network 

across 5 selling locations 

to meet with demand by location at the lowest cost. 

You can decide the plant size in each location where the options are low capacity and high capacity. 

One possibility is to set up a facility in each region with an advantage of low transportation costs and a disadvantage of having production plans sized to meet local demands and not exploiting the economies of scale. 

Another possibility is to set up a few manufacturing plants with an advantage of economies of scale but requiring higher transportation costs. 


Given, you have the estimated demand for each location, 

variable costs of transportation from one plant to another, 

fixed costs of having a plant is based on its size 

and the production capacity is based on plant size where 500 for low capacity and 1500 for high capacity. 

How would you solve this problem with the minimum cost?


        Demand	A	B	C	D	E	Hich_C	Low_C
        
    A	145.40	08	14	21	21	12	4730	3230

    B	84.100	14	6	13	14	13	7270	4980

    C	156.40	23	13	08	10	22	3080	2110

    D	1676.8	21	14	09	03	20	9100	6230

    E	2719.6	12	13	17	20	06	9500	6500
    

Demand column shows the estimated demand for each location

Columns A to E shows the transportation cost from locations in the index to each of the locations in columns (e.g. transportation cost from plant B to plant D is 14)

‘High_C’ and ‘Low_C’ columns show the fixed cost of having high and low capacity plants in each location (i.e. having a low capacity plant at location E has AUD 6500 fixed cost)

### Solution

To solve this problem, we first need to initialize our model and decision variables. There are two decision variables;
   1. Production quantity that is produced in location i and shipped to location j (continuous variable)
   2. Capacity of the production plant i (binary variable: 1 if the plant at location i of capacity s is open, 0 if closed)
   
   

### Implementation

In [7]:
# import required libraries
import pulp
from pulp import *
import pandas as pd

C:\Users\User\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\User\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [25]:
# read fix cost and variable cost from the file
df = pd.read_csv('Production.txt', sep = '\t')
df

,Loc,Demand,A,B,C,D,E,High_C,Low_C
0,A,145.4,8,14,21,21,12,4730,3230
1,B,84.1,14,6,13,14,13,7270,4980
2,C,156.4,23,13,8,10,22,3080,2110
3,D,1676.8,21,14,9,3,20,9100,6230
4,E,2719.6,12,13,17,20,6,9500,6500


In [2]:
# 1. Initialize model
'''
Inside LpProblem() method we define the problem name and sense of objective function 
that can either be ‘LpMaximize’ or ‘LpMinimize’. 
In this example we want to minimize number production cost, so LpMinimize.
'''

model = LpProblem('Location Selection', LpMinimize)

C:\Users\User\anaconda3\lib\site-packages\pulp\pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [5]:
# 2. Define Decision Variables
'''
Inside LpVariable() method we define a name for the variable, values for lower and upper bound, 
and category type which can be ‘Integer’, ‘Binary’, or ‘Continuous’. 
Since we want an integer value for the number of workers, we choose integer.
'''

#decision variable list
loc = ['A', 'B', 'C', 'D', 'E']
size = ['Low_C','High_C']

# Production quantity that is produced in location i and shipped to location j (continuous variable)
x = LpVariable.dicts('production_', [(i,j) for i in loc for j in loc], lowBound=0, upBound=None, cat='Continuous')
#Capacity s of the production plant i (binary variable: 1 if the plant at location i of capacity s is open, 0 if closed)
y = LpVariable.dicts('plant_', [(i,s) for s in size for i in loc], lowBound=None, upBound=None, cat='Binary')

In [21]:
# 3. Define the Objective Function
'''
Add the objective function to the initialized model using+=
As we have a dictiony of dicision variables, we can use lpSum to take the sum. 
'''

# extract fix cost and variable cost
fix_cost = df[size]
fix_cost.index = loc
var_cost = df[loc]
var_cost.index = loc

model += (lpSum([fix_cost.loc[i,s] * y[(i,s)] for s in size for i in loc]) + 
          lpSum([var_cost.loc[i,j] * x[(i,j)] for i in loc for j in loc]))

In [30]:
var_cost

,A,B,C,D,E
A,8,14,21,21,12
B,14,6,13,14,13
C,23,13,8,10,22
D,21,14,9,3,20
E,12,13,17,20,6


In [33]:
# 4. Define the Constraints
'''
Constraints are:
1. Total production needs to be equal to total demand
2. Total production can be smaller or equal to total production capacity
'''

# extract demand 
demand = df[['Demand']]
demand.index = loc

# creat capacity from the problem statement
capacity = pd.DataFrame({size[0]:[500 for i in loc], size[1]:[1500 for i in loc]})
capacity.index = loc

# Total production needs to be equal to total demand
for j in loc:
    model += lpSum([x[(i, j)] for i in loc]) == demand.loc[j,'Demand'] 
    
# Total production can be smaller or equal to total production capacity
for i in loc:
    model += lpSum([x[i, j] for j in loc]) <= lpSum([capacity.loc[i,s] * y[i,s] for s in size])

In [34]:
# 5. Solve Model
# the method returns 1 if the model is solvable
model.solve()

1

In [35]:
# Print model status
print('Status:', LpStatus[model.status])

Status: Optimal


In [36]:
#Print solution of optimized decision variables
for variable in model.variables():
    print ('{} = {}'.format(variable.name, variable.varValue))

plant__('A',_'High_C') = 1.0
plant__('A',_'Low_C') = 0.0
plant__('B',_'High_C') = 0.0
plant__('B',_'Low_C') = 0.0
plant__('C',_'High_C') = 0.0
plant__('C',_'Low_C') = 1.0
plant__('D',_'High_C') = 1.0
plant__('D',_'Low_C') = 0.0
plant__('E',_'High_C') = 1.0
plant__('E',_'Low_C') = 0.0
production__('A',_'A') = 145.4
production__('A',_'B') = 0.0
production__('A',_'C') = 0.0
production__('A',_'D') = 0.0
production__('A',_'E') = 1219.6
production__('B',_'A') = 0.0
production__('B',_'B') = 0.0
production__('B',_'C') = 0.0
production__('B',_'D') = 0.0
production__('B',_'E') = 0.0
production__('C',_'A') = 0.0
production__('C',_'B') = 84.1
production__('C',_'C') = 156.4
production__('C',_'D') = 176.8
production__('C',_'E') = 0.0
production__('D',_'A') = 0.0
production__('D',_'B') = 0.0
production__('D',_'C') = 0.0
production__('D',_'D') = 1500.0
production__('D',_'E') = 0.0
production__('E',_'A') = 0.0
production__('E',_'B') = 0.0
production__('E',_'C') = 0.0
production__('E',_'D') = 0.0
produc

In [39]:
# Results for production quantities
[{'prod':'{} to {}'.format(i,j), 'quantity':x[(i,j)].varValue} 
     for i in loc for j in loc]

[{'prod': 'A to A', 'quantity': 145.4},
 {'prod': 'A to B', 'quantity': 0.0},
 {'prod': 'A to C', 'quantity': 0.0},
 {'prod': 'A to D', 'quantity': 0.0},
 {'prod': 'A to E', 'quantity': 1219.6},
 {'prod': 'B to A', 'quantity': 0.0},
 {'prod': 'B to B', 'quantity': 0.0},
 {'prod': 'B to C', 'quantity': 0.0},
 {'prod': 'B to D', 'quantity': 0.0},
 {'prod': 'B to E', 'quantity': 0.0},
 {'prod': 'C to A', 'quantity': 0.0},
 {'prod': 'C to B', 'quantity': 84.1},
 {'prod': 'C to C', 'quantity': 156.4},
 {'prod': 'C to D', 'quantity': 176.8},
 {'prod': 'C to E', 'quantity': 0.0},
 {'prod': 'D to A', 'quantity': 0.0},
 {'prod': 'D to B', 'quantity': 0.0},
 {'prod': 'D to C', 'quantity': 0.0},
 {'prod': 'D to D', 'quantity': 1500.0},
 {'prod': 'D to E', 'quantity': 0.0},
 {'prod': 'E to A', 'quantity': 0.0},
 {'prod': 'E to B', 'quantity': 0.0},
 {'prod': 'E to C', 'quantity': 0.0},
 {'prod': 'E to D', 'quantity': 0.0},
 {'prod': 'E to E', 'quantity': 1500.0}]

In [40]:
# Results for plant capacities based on location
[{'lowCap':y[(i,size[0])].varValue, 'highCap':y[(i,size[1])].varValue}
     for i in loc]

[{'lowCap': 0.0, 'highCap': 1.0},
 {'lowCap': 0.0, 'highCap': 0.0},
 {'lowCap': 1.0, 'highCap': 0.0},
 {'lowCap': 0.0, 'highCap': 1.0},
 {'lowCap': 0.0, 'highCap': 1.0}]

In [41]:
# Print optimized objective function value(model.objective)

print('Objective = ', value(model.objective))

Objective =  58850.9


As can be seen from results, the model suggests to open a low capacity plant in location C, and high capacity plants in locations A, D, and E. The demand at location B is suggested to be provided by the location C. Doing so, we can achieve the minimum cost of $58850.9.

In [38]:
# P.N. Some implementation details are missing in the reference
# Reference: https://towardsdatascience.com/how-to-develop-optimization-models-in-python-1a03ef72f5b4